In [1]:
pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/a9/cc/f2bbce0ad52e09cd1aecb724af06385021b42a7317cd5938ba9c8581509d/openai-1.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/2c/93/13f25f2f78646bab97aee7680821e30bd85b2ff0fc45d5fdf5393b79716d/httpcore-1.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for h11<0.15,>=0.13 from https://files.pythonhosted.org/packages/95/04/ff642e65ad6b9

In [1]:
import os
import openai

In [2]:
pip install cohere tiktoken

  Obtaining dependency information for cohere from https://files.pythonhosted.org/packages/a4/49/6f9cae01e248bef4238525728cca006ca375eebc5874e6f246907f9baec9/cohere-4.53-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/9e/11/83ca4e19bb6fc15971e543725ae1269a8a1c133e55b5952801ab9c0bcc9e/tiktoken-0.6.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for backoff<3.0,>=2.0 from https://files.pythonhosted.org/packages/df/73/b6e24bd22e6720ca8ee9a85a0c4a2971af8497d8f3193fa05390cbd46e09/backoff-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for fastavro<2.0,>=1.8 from https://files.pythonhosted.org/packages/45/35/ed6a09aa1bcaf3cf49f158ff6b7fa91cabe070f10e5d4f0618af8614c8c0/fastavro-1.9.4-cp311-cp311-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 2.9 MB/

In [2]:
import tiktoken

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

open.api_key = os.environ['OPENAI_API_KEY']

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt}]
  response = openai.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0,
  )
  return response.choices[0].message.content

In [5]:
response = get_completion("What is the capital of France?")

In [6]:
print(response)

The capital of France is Paris.


In [7]:
def get_prompt(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens=500,
    )
    return response.choices[0].message.content

with open("dataset.txt") as phrases_file, open("no_cue_prompts.txt") as prompts_file:
    phrases = phrases_file.readlines()
    prompts = prompts_file.readlines()
        
    for prompt in prompts: 
        generated_text = get_prompt(prompt)
        print(f"No Cue Prompt: {prompt}")
        print(f"Generated Text: {generated_text}\n")

Original Prompt: The employees knew the contract

Generated Text:  inside and out, as they had spent hours reviewing and discussing it during their training sessions. They were well-versed in the terms and conditions, and were able to confidently answer any questions or concerns that arose from clients or colleagues. Their thorough understanding of the contract helped to ensure that all parties involved were on the same page and that the agreement was carried out smoothly and efficiently.

Original Prompt: The mechanic accepted the car

Generated Text:  and began inspecting it for any issues. He checked the engine, brakes, tires, and other components to ensure everything was in working order. After completing the inspection, he provided a list of recommended repairs and maintenance services to the car owner. The owner agreed to have the work done, and the mechanic got to work fixing the car. Once the repairs were completed, the car was returned to the owner in good condition, ready to 

Original Prompt: The reporter revealed the politician

Generated Text: The reporter revealed that the politician had been involved in a corruption scandal, accepting bribes in exchange for political favors. The politician denied the allegations, but the evidence presented by the reporter was damning. The story sent shockwaves through the political world, leading to calls for the politician to resign and face legal consequences for their actions.

Original Prompt: The owners insured the house

Generated Text:  against fire, theft, and other potential damages.

Original Prompt: The lawyer established the alibi

Generated Text: for his client by presenting evidence that showed the client was out of town at the time of the crime.

Original Prompt: The store guaranteed the television
Generated Text:  for one year against any defects or malfunctions.

